# GPT model prompting through Azure OpenAI

Model predictions are saved into a JSONL-file

In [ ]:
import json
import re
import time
import openai

In [ ]:
# Formatted prompts containig instructions and text (expected answer is removed during prompting)
test_file = ""

# Results folder
res_path = ""
# Results filename
resultfilename = ""

# API key file location
api_key_file = ""

In [ ]:
def write_examples_to_file(filename, examples):
    '''
    The function writes the model's responses to a JSONL file.

    The function takes in formatted model responses (examples) and writes them into the given file (filename).
    '''
    
    with open(filename, "a", encoding='utf-8') as file:
        for example in examples:
            json.dump(example, file, ensure_ascii=False)
            file.write('\n')
            

## API teave

In [ ]:
print(openai.__version__) # 0.28.0 

In [ ]:
# Azure API information
openai.api_type = "azure"

# API key
with open(api_key_file, 'r') as api_file:
    openai.api_key=str(api_file.readline()).strip()
    
openai.api_base = "" # API base
openai.api_version = "" #API version

## Mudeli viipamine

In [ ]:
prompts = []
responses = [] 

In [ ]:
def GPT_predict(system, prompt):
    """
    The function prompts models through the Azure API, which returns the model's response. 
    In case of an error, the model returns None.
    
    The function parameters are the system prompt (system) and the model prompt (prompt)
    """

    try: 
        # Viipan mudelit
        response = openai.ChatCompletion.create(
            deployment_id = "",
            model = "gpt-35-turbo", # Left as an example
            messages=[
                {"role": "system", "content": system},
                {"role": "user", "content": prompt}
            ]
        )
    
        responses.append(response['choices'][0]['message']['content'])
        prompts.append(prompt)

        return response['choices'][0]['message']['content']
    
    except Exception as e:
        return None

In [ ]:
# Formatted prompts
with open(test_file, 'r', encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]

# A list for storing all incorrectly formatted model responses.
predictions = []

# A list for storing all correctly formatted model responses.
results = []

In [ ]:
# Due to the tokens-per-minute limit, the examples were divided into groups of 5 
# to fit within the 21k token limit (set during deployment).

intervals = []

for i in range(35, len(dataset), 5):
    n = i+5
    if n > len(dataset):
        n = len(dataset)
    intervals.append((i, n))

print(intervals)

In [ ]:
failed = 0

# The model is prompted with each example
for interval in intervals:
    print(interval)

    examples = []

    for i, message in enumerate(dataset[interval[0]:interval[1]]):
        
        example = {}
        
        system_t = message["messages"][0]["content"]
        text = message["messages"][1]["content"]
        test_labels = message["messages"][2]["content"]
    
        added = False

        # Prompting the model with a single prompt a maximum of 2 times
        for j in range(2):
            # In case of an error, the model is prompted again with the same prompt
            if not added:
                predicted_labels = GPT_predict(system_t, text)

                if predicted_labels:
                    # Replace single quotation marks surrounding strings with double quotation marks
                    pattern = r"(?<=\W)'|'(?=\W)"
                    predicted_labels = re.sub(pattern, '"', predicted_labels)

                    # Check if the model returned the response in correct JSON format
                    try:
                        example["original"] = test_labels
                        example["predicted"] = json.loads(predicted_labels)
                        
                        results.append(example)
                        examples.append(example)
                        
                        added = True
                        
                    except Exception as error:
                        failed += 1
                        predictions.append(predicted_labels)

    # Writing the existing responses into the results file
    write_examples_to_file(res_path+resultfilename, examples)
    
    # 1 minute wait between groups to stay within the limit
    time.sleep(60)

print("Number of results:", len(results))
print("Number of failed examples:", len(dataset) - len(results))
print("Number of fixed examples:", failed - 2*(len(dataset) - len(results)))